<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Decisions with AI Agent Personas

Empower multi-agent systems where specialists are defined to execute requests they were designed to fulfill.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.33.0" 
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;

var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName!,
        endpoint: azureOpenAIEndpoint!,
        apiKey: azureOpenAIAPIKey!)
    .Build();

Installed Packages Microsoft.SemanticKernel, 1.33.0 Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.33.0-alpha

### Step 2 - Creating an AI Agent Decision Persona

Personas leverage the Chat Completion service from the Semantic Kernel Agents package. Personas are basically prompt instructions on how the GenAI persona should behave, respond, expertise information and the tone it should respond. These personas can be instructed to also use specific decision-making capabilities. In more technical terms, AI Agent personas introduce "bias" specificity to the Generative AI model. 

> Note: The term **bias** in statistics, machine learning and AI is not a "bad" word. Instead, it's a technical term that refers to how a model or an estimate might simplify assumptions or alter internal data distributions to make predictions. For example, an AI model may be instructed to approach a problem from a perspective of a football player or be trained with mostly football data. This would be a "highly biased" model skewed towards the football perspective.  

In this example, the agent will take on the persona of **Dwight Eisenhower**. In addition for being famous as a leader of allied forces in WW2 and a US president, Dwight Eisenhower is famous of the "Eisenhower Matrix" decision framework.  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceFramework/DecisionIntelligence-Example-EisenhowerPrioritizationMatrix.png">  

> "In preparing for battle, I have always found that plans are useless but planning is indispensable."
>
> -- <cite>Dwight Eisenhower (WW2 Allied Leader, US President)</cite> 

The Eisenhower Matrix Priorotization Framework is almost 80 years old. However, it is still actively used in current software systems. For example, Asana (SaaS project management software) uses the Eisenhower Matrix. https://asana.com/resources/eisenhower-matrix  
<img style="display: block; margin: auto;" width ="700px" src="https://assets.asana.biz/transform/a4725133-d094-48e1-9e6d-f65c36f0cd59/inline-leadership-eisenhower-matrix-1-2x?io=transform:fill,width:2560&format=webp">  

Executing the code below will ask multiple decision intelligence questions to the single agent (persona) of Dwight Eisenhower. The decision recommendations will come from an AI perspective of a Dwight Eisenhower persona. This persona is crafted with two simple instructions:
* Approach the problem like Dwight Eisenhower.  
* Use a systematic decision-making framework "Eisenhower Decision Matrix"
* If using the "Eisenhower Decision Matrix", output in a nice Markdown table format  

In [ ]:
#pragma warning disable SKEXP0110
// Helper method to invoke the chat completion agent with a list of prompts
private static async Task InvokeChatCompletionAgentWithTemplateAsync(ChatCompletionAgent agent, string[] prompts)
{
    ChatHistory chat = [];

    foreach (var prompt in prompts)
    {
        // Add input to chat
        ChatMessageContent request = new(AuthorRole.User, prompt);
        chat.Add(request);
        Console.WriteLine(request);

        KernelArguments? arguments = null;

        // Process agent response
        await foreach (ChatMessageContent message in agent.InvokeAsync(chat, arguments))
        {
            chat.Add(message);
            Console.WriteLine(message);
        }
    }
}
#pragma warning restore SKEXP0110

// Notice the agent is created with a name, instructions, and description
var name = "Dwight Eisenhower";
var instructions = """
You are a decision intelligence strategist. Provide structured, logical, and comprehensive advice.
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Approach the problem how Dwight Eisenhower approached strategic problems and 
when appropriate using the Dwight Eisenhower's Decision Prioritization Matrix.

If using the Decision Prioritization Matrix, output it in a Markdown table format.
""";
var description = "A chat bot that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.";

string[] prompts = 
[
"How should I prioritize my typical unproductive day?",
"I am looking to buy a new car, what should I consider before making a decision?",
];

#pragma warning disable SKEXP0110
            // Instruction based template always processed by KernelPromptTemplateFactory
            ChatCompletionAgent agent =
                new()
                {
                    Kernel = semanticKernel,
                    //Name = name,
                    Description = description,
                    Instructions = instructions
                };

            await InvokeChatCompletionAgentWithTemplateAsync(agent, prompts);
#pragma warning restore SKEXP0110

How should I prioritize my typical unproductive day?
It sounds like you're looking to optimize the structure of your day, identify your most critical tasks, and eliminate or delegate things that waste time. In the spirit of Dwight Eisenhower's prioritization approach, let's tackle this systematically.

### Step 1: Reflect and Identify
You'll begin by listing all the common tasks and activities you engage in during your "unproductive days." Ask yourself:

1. **What tasks consistently consume my time?**
2. **Which tasks are actually important or beneficial to me (even if they don't feel urgent)?**
3. **What activities lead to little or no value? Are there distractions stealing my time?**
4. **Are there areas where I procrastinate or feel overwhelmed?**

Once you have your list of activities, we'll categorize them using the **Eisenhower Decision Prioritization Matrix**.

---

### Step 2: Apply the Eisenhower Matrix

Below is the **Eisenhower Matrix** approach, dividing tasks into four cat

Example table output using the **"Eisenhower Prioritization Matrix"** to the question: **"How should I prioritize my typical unproductive day?"**: 

| Urgent/Important         | Important/Not Urgent          | Urgent/Not Important       | Not Urgent/Not Important    |
|--------------------------|-------------------------------|----------------------------|-----------------------------|
| Prepare for a meeting    | Reading a book                | Answering non-critical emails | Watching TV                  |
| Paying bills             | Exercise                      | Responding to non-urgent messages | Browsing social media         |
| Fixing a crucial work error| Planning out weekly meals   | Running a non-critical errand | Playing games                |


Example table output using the **"Eisenhower Prioritization Matrix"** to the question: **"I am looking to buy a new car, what should I consider before making a decision?"**: 

| Urgent/Important                           | Important/Not Urgent                     | Urgent/Not Important                | Not Urgent/Not Important           |
|--------------------------------------------|------------------------------------------|-------------------------------------|------------------------------------|
| **Budgeting and Financing**                | **Long-Term Ownership Costs**            | **Dealership Promotions**           | **Additional Luxury Features**      |
| **Primary Usage Requirements**             | **Resale Value and Depreciation**        | **Brand Loyalty Offers**            | **Cosmetic Preferences**            |
| **Safety Features**                        | **Warranty and Service**                 | **Short-Term Insurance Discounts**  | **Optional Upgrades**               |
| **Test Driving Suitable Models**           | **Environmental Impact Assessment**      | **Accessories Given**               | **Color Choice**                    |


### Step 3 - Creating a Multi-Agent Decision Personas 

> "The many are smarter than the few when it comes to solving problems, predicting outcomes, and innovating solutions, as long as there is diversity of opinion, independence, and decentralization."  
>
> -- <cite>James Surowiecki (Author "The Wisdom of Crowds")</cite> 

Single agent personas are effective constructs in Semantic Kernel, but can largely be replicated in other mechanisms with prompts. **Creating multiple personas (experts) that can interact with each other is the true power of the Semantic Kernel Agent functionality. This allows multiple chat agents to interact in various ways.** 

Execute the cell below to illustrate how two different personas can interact in a decision-making process moderated by a decision intelligence expert that makes the final reasoning conclusiion on the decision task at hand. In the output notice key steps:
* Each expert (Dwight and Ben) make their recommendation, based on their decision persona
* The final recommendation is made based on the synthesis of the agent recommendations  
* This multi-agent interaction to converge on a single decision mimic the self-consistency GenAI pattern 

Note: The execution below can take the some time. Use GPT-4o-mini to optimize performance speed of the execution.  

In [5]:
using System.Threading;

#pragma warning disable SKEXP0110
// Create a new chat TerminationStrategy that will terminate when the final message contains the term "final decision"
private sealed class ApprovalTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "approve"
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
        => Task.FromResult(history[history.Count - 1].Content?.Contains("final decision", StringComparison.OrdinalIgnoreCase) ?? false);
}

var assistantDwightEisenhower = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    //Name = "Dwight", // Dwight Eisenhower persona
    Description = "A chat bot that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantBenFranklin = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    //Name = "Ben", // Ben Franklin persona
    Description = "A chat bot that replies to the message with recommending the Ben Franklin Decision Framework.",
    Instructions = "You are a decision intelligence strategist with the knowledge of Ben Franklin's Decision Framework. Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. Provide structured, logical, and comprehensive advice."
};

var assistantDecisionMaker = new ChatCompletionAgent
{
    Kernel = semanticKernel,
    //Name = "DecisionMaker",
    Description = "A chat bot that approves the final decision.",
    Instructions = """
    You are a Decision Intelligence maker. Make the final decision and describe the reasoning from the recommendations.
    You're laser focused on the goal at hand. Don't waste time with chit chat.
    Consider suggestions when refining an idea. Use the words final decision to approve the final decision.
    """
};

// Create a chat for agent interaction.
AgentGroupChat chat =
    new(assistantDwightEisenhower, assistantBenFranklin, assistantDecisionMaker)
    {
        ExecutionSettings =
            new()
            {
                // Here a TerminationStrategy subclass is used that will terminate when
                // an assistant message contains the term "approve".
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        // Only the art-director may approve.
                        Agents = [assistantDecisionMaker],
                        // Limit total number of turns
                        MaximumIterations = 10,
                    }
            }
    };

// Invoke chat and display messages.
ChatMessageContent input = new(AuthorRole.User, """
Decision: Buy a house for a rental property or invest in Bitcoin? 
Constraint: You want to optimize for long-term financial growth, but also mitigate risk. 
""");
chat.AddChatMessage(input);
Console.WriteLine(input);
Console.WriteLine();

await foreach (ChatMessageContent response in chat.InvokeAsync())
{
    Console.WriteLine(response);
}

Console.WriteLine($"\n[IS COMPLETED: {chat.IsComplete}]");

Decision: Buy a house for a rental property or invest in Bitcoin? 
Constraint: You want to optimize for long-term financial growth, but also mitigate risk. 

To help you make an informed decision between buying a house for a rental property or investing in Bitcoin, let's use Dwight Eisenhower's Decision Prioritization Matrix and also consider a few essential factors, such as risk, potential return, and effort required. We'll address each option in turn.

### Decision Prioritization Matrix
The Eisenhower Matrix classifies tasks based on their urgency and importance, into four quadrants:

1. **Urgent and Important**: Tasks you will do immediately.
2. **Not Urgent but Important**: Tasks to schedule for later.
3. **Urgent but Not Important**: Tasks to delegate.
4. **Not Urgent and Not Important**: Tasks to eliminate.

For financial decisions like buying property or investing in Bitcoin, the matrix can be adapted to weigh urgency, importance, risk, and effort.

### Option 1: Buying a House 